# Adam's Project - Segmentation



 This notebook contains the code used for running DAPI segmentation. 

 * path: /mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/*zarr
 * microscope: BBScope4
 * Experiment: WTC 11 Combinatorial data


# Get the files associated with the experiment

 define a segmentation folder, max_image, etc by refering to CellSegmentationClean.ipynb

In [1]:
import glob
import sys

from ioMicro import *
path = r'/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/*zarr'
files = glob.glob(path)
print(f"total FOVs: {len(files)}")

total FOVs: 211


getting the folders here:

In [2]:
import os

analysis_folder =r'/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/AnalysisDeconvolve_CG'
segm_folder = r'/home/cfg001/Desktop/wtc/segmentation'


In [3]:
import napari
napari.__version__

'0.4.18'

In [7]:
!pip install typing-extensions==4.5.0 

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
filelock 3.12.3 requires typing-extensions>=4.7.1; python_version < "3.11", but you have typing-extensions 4.5.0 which is incompatible.


In [9]:
import numpy as np
psf = np.load(r'/home/cfg001/Desktop/wtc/CT/psfs/psf_750_Scope4_final.npy')
import napari
lol = napari.view_image(psf)

Traceback (most recent call last):
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 502, in mousePressEvent
    self._vispy_mouse_press(
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/vispy/app/base.py", line 184, in _vispy_mouse_press
    ev = self._vispy_canvas.events.mouse_press(**kwargs)
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/napari/_vispy/canvas.py", line 97, in _process_mouse_event
    super()._proc

In [ ]:
!pip uninstall napari
!y


Found existing installation: napari 0.4.17
Uninstalling napari-0.4.17:
  Would remove:
    /home/cfg001/miniconda3/envs/cellpose/bin/napari
    /home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/napari-0.4.17.dist-info/*
    /home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/napari/*
    /home/cfg001/miniconda3/envs/cellpose/lib/python3.9/site-packages/napari_builtins/*
Proceed (Y/n)? 

In [9]:
files

['/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__119.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__152.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__036.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__094.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__007.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__163.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__128.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__065.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__101.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__130.zarr',
 '/mnt/Ren-Imaging_NAS/Adam/E200_WTC11_WTday3__8_31_2023/H1_DMER_1/Conv_zscan__054.zarr',
 '/mnt/Ren

In [10]:
def compute_mask(model,file,segm_folder,cell_diameter=20,psf = np.load(r'/home/cfg001/Desktop/wtc/CT/psfs/psf_750_Scope4_final.npy'),
                 deconvolve=True,save=True,max_image = 10000,snorm=0,
                 flat_field_fl = r'/home/cfg001/Desktop/wtc/CT/flat_field/Scope4_med_col_raw3.npz',redo=False):
    if not os.path.exists(segm_folder): os.makedirs(segm_folder)
    save_fl  = segm_folder+os.sep+os.path.basename(file).split('.')[0]+'--'+os.path.basename(os.path.dirname(file))+'--CYTO_segm.npz'
    print("Working on : " + save_fl)
    masks,tensor,cast = None,None,None
    if not os.path.exists(save_fl) or redo:
        #load image
        load = read_im(file)
        #cast image
        cast = np.array(load[-1],dtype=np.float32)#####mistake
        ### Flat field correct
        im_med = np.load(flat_field_fl)['im']
        im_med = cv2.blur(im_med,(20,20))
        cast = cast/im_med*np.median(im_med)
        tensor = cast
        if deconvolve:
            #using psf and getting tensor
            #tensor = full_deconv(cast,s_=500,pad=100,psf=psf,parameters={'method': 'lucy',  'niter': 30},gpu=True,force=False)
            tensor = full_deconv(cast,s_=500,pad=100,psf=psf,parameters={'method': 'wiener',  'beta': 0.01},gpu=True,force=False)
        if snorm>0:
            tensor = norm_slice(cast,snorm)
        #running the cellpose:
        #step 4 is running the cell i was working with last week.
        use_gpu = True
        def blur(tensor,s=100):
            return np.array([cv2.blur(im_,(s,s)) for im_ in tensor],dtype=np.float32)
        from cellpose import models,io,utils
        model = models.Cellpose(gpu=use_gpu, model_type=model)
        #img = tensor[:,::4,::4]
        #img = blur(tensor,2)[::4,::4,::4]
        img = tensor[::4,::4,::4]
        img_final = np.clip(img/max_image,0,1)
    
        masks, flows, styles, diams = model.eval(img_final,z_axis=0, diameter=cell_diameter, channels=[0,0],do_3D=False,normalize=False,flow_threshold=0.75,cellprob_threshold=-5,stitch_threshold=0.5) 
    
        shape = np.array(tensor.shape)
        if save:
            #step 5: saving the mask and shape in the appropriate location
            np.savez_compressed(save_fl,segm = masks,shape = shape,tensor = tensor)
    return masks,tensor,cast

In [11]:
file = files[180]
masks,tensor,cast = compute_mask("nuclei",file,segm_folder,cell_diameter=20,
                                 deconvolve=True,
                                 save=True,
                                 max_image = 11000,snorm=0,redo=True)

Working on : /home/cfg001/Desktop/wtc/segmentation/Conv_zscan__028--H1_DMER_1--CYTO_segm.npz


100%|███████████████████████████████████████████| 36/36 [00:04<00:00,  8.10it/s]


In [13]:
file = np.load(r'/home/cfg001/Desktop/wtc/segmentation/Conv_zscan__028--H1_DMER_1--CYTO_segm.npz')
file.files

['segm', 'shape', 'tensor']

### Testing

In [26]:
use_gpu = True
max_image = 11000
def blur(tensor,s=100):
    return np.array([cv2.blur(im_,(s,s)) for im_ in tensor],dtype=np.float32)
from cellpose import models,io,utils
model = 'nuclei'
model = models.Cellpose(gpu=True, model_type=model)
#img = tensor[:,::4,::4]
#img = blur(tensor,2)[::4,::4,::4]
img = tensor[::4,::4,::4]
img_final = np.clip(img/max_image,0,1)

masks, flows, styles, diams = model.eval(img_final,z_axis=0, diameter=20, channels=[0,0],do_3D=False,normalize=False,flow_threshold=0.75,cellprob_threshold=-5,stitch_threshold=0.5) 

In [27]:
import napari
V = napari.view_image(cast)
V.add_image(tensor,contrast_limits=[0,11000])
V.add_labels(resize(masks,tensor.shape))

<Labels layer 'Labels' at 0x222b48ae880>

In [33]:
file = files[180]
napari.view_image(read_im(file)[-1])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1023.5, 1023.5), zoom=0.2583740234375, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(25.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 50.0, 1.0), (0.0, 2048.0, 1.0), (0.0, 2048.0, 1.0)), current_step=(25, 1024, 1024), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x223447e4e20>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.T

In [37]:
np.percentile(tensor,99)

4772.5683740234235

In [14]:
for file in tqdm(files):
    masks,tensor,cast = compute_mask("nuclei",file,segm_folder,cell_diameter=20,
                                 deconvolve=True,
                                 save=True,
                                 max_image = 11000,snorm=0,redo=True)

  0%|                                                   | 0/211 [00:00<?, ?it/s]

Working on : /home/cfg001/Desktop/wtc/segmentation/Conv_zscan__119--H1_DMER_1--CYTO_segm.npz



  0%|▏                                        | 1/211 [01:20<4:40:36, 80.17s/it]

Working on : /home/cfg001/Desktop/wtc/segmentation/Conv_zscan__152--H1_DMER_1--CYTO_segm.npz



  1%|▍                                        | 2/211 [02:50<4:59:03, 85.86s/it]

...


 99%|██████████████████████████████████████▋| 209/211 [5:25:43<03:06, 93.05s/it]

Working on : /home/cfg001/Desktop/wtc/segmentation/Conv_zscan__117--H1_DMER_1--CYTO_segm.npz



100%|██████████████████████████████████████▊| 210/211 [5:27:16<01:32, 92.91s/it]

Working on : /home/cfg001/Desktop/wtc/segmentation/Conv_zscan__038--H1_DMER_1--CYTO_segm.npz



100%|███████████████████████████████████████| 211/211 [5:29:04<00:00, 93.57s/it]


In [ ]:
mask = r'\\merfish8\merfish8v2\20230801D104Myh67d80\DNA_singleCy5\AnalysisDeconvolveCG\SegmentationDAPI_CG\Conv_zscan__001--H1_R1--CYTO_segm.npz'

image = read_im(r"\\merfish8\merfish8v2\20230801D104Myh67d80\DNA_singleCy5\")